# Use VMCON solution as initial point for no-f-value run

## VMCON solution as initial point for original f-value IN.DAT

First, take VMCON local DEMO solution. Create an input file from it, and check it works. Does the output solution work as an input?

In [2]:
# Reload Process each time (keep editable install up-to-date)
%load_ext autoreload
%autoreload 2

from process.io import plot_solutions as ps
from pathlib import Path
import process
from process.io.mfile import MFile
from process.io.in_dat import InDat

In [3]:
# Define data location
vmcon_mfile = Path("../demo_vmcon_MFILE.DAT")

# Create Mfile object
mfile = MFile(vmcon_mfile)

# Get number of optimisation parameters
n = int(mfile.data["nvar"].get_scan(-1))

# Get all n optimisation parameter names and values from "itvarxxx" number
opt_params = {}
for i in range(n):
    opt_param_no = f"itvar{i+1:03}"
    param_name = mfile.data[opt_param_no].var_description
    param_value = mfile.data[opt_param_no].get_scan(-1)
    opt_params[param_name] = param_value

# Print optimisation parameters at global solution
print(f"VMCON solution = {opt_params}")

VMCON solution = {'beta': 0.044039, 'dene': 6.8094e+19, 'fwalld': 0.56327, 'ffuspow': 1.02, 'ftburn': 1.0, 'flhthresh': 1.575, 'fpinj': 0.2, 'fpnetel': 0.71469, 'fbetatry': 0.63976, 'fpeakb': 0.7254, 'coheof': 9102000.0, 'fjohc': 0.058528, 'fjohc0': 0.072846, 'fcohbop': 1.0, 'fiooic': 0.091059, 'fvdump': 1.0, 'vdalw': 10.0, 'fjprot': 1.0, 'ftmargtf': 0.24667, 'ftmargoh': 0.29975, 'ftaulimit': 1.0, 'ftaucq': 0.77584, 'foh_stress': 1.0, 'fbmaxcs': 0.77812, 'fne0': 0.48113, 'fpsepbqar': 1.0, 'fstrcase': 1.0, 'fstrcond': 0.85914, 'bt': 4.7036, 'rmajor': 9.6829, 'te': 14.567, 'hfact': 1.2, 'tfcth': 1.4, 'ohcth': 0.92066, 'q': 3.0, 'bore': 2.8236, 'fvsbrnni': 0.001, 'tdmptf': 27.632, 'thkcas': 0.2194, 'thwcndut': 0.008, 'fcutfsu': 0.5, 'cpttf': 65000.0, 'ralpne': 0.090682, 'oh_steel_frac': 0.27702, 'fimp(13)': 0.00081666, 'dr_tf_wp': 0.57792, 'fgwped': 0.72}


Now write a new input file with the original solution as the initial optimisation parameters.

In [4]:
# Now read in original IN.DAT
in_dat_path = Path("../demo_IN.DAT")
in_dat = InDat(in_dat_path)

# Change to the new optimisation parameter values
for var_name, value in opt_params.items():
    in_dat.add_parameter(var_name, value)

# Write out new IN.DAT, with optimisation parameters set to previous local solution
new_in_dat_path = Path("demo_vmcon_soln_IN.DAT")
in_dat.write_in_dat(new_in_dat_path)

In [5]:
single_run = process.main.SingleRun(str(new_in_dat_path), solver="scipy")
single_run.run()

The IN.DAT file does not contain any obsolete variables.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 2.4.0   Release Date :: 2022-05-18
   Tag No. : v2.4.0-963-gcda1a8011 code contains untracked changes
    Branch : nlopt-soln-plotting
   Git log : Use iter var bounds in input.f90
 Date/time : 21 May 2023 15:26:04 +01:00(hh:mm) UTC
      User : jon
  Computer : jon-Precision-3560
 Directory : /home/jon/code/notebooks/global_opt/no_f_value_initial_test
     Input : 

/home/jon/.local/lib/python3.10/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Evaluation 1365, objective function = -2.2143, constraint residuals = 5.945e+03
Evaluation 1366, objective function = -1.632, constraint residuals = 1.232e+00
Evaluation 1367, objective function = -1.5156, constraint residuals = 8.295e-01
Evaluation 1368, objective function = -1.4543, constraint residuals = 4.521e-01
Evaluation 1369, objective function = -1.4252, constraint residuals = 2.187e-01
Evaluation 1370, objective function = -1.412, constraint residuals = 9.921e-02
Evaluation 1371, objective function = -1.4062, constraint residuals = 4.355e-02
Evaluation 1372, objective function = -1.4037, constraint residuals = 1.880e-02
Evaluation 1373, objective function = -1.4026, constraint residuals = 8.015e-03
Evaluation 1374, objective function = -1.4022, constraint residuals = 3.369e-03
Evaluation 1375, objective function = -1.402, constraint residuals = 1.391e-03
Evaluation 1376, objective function = -1.4005, constraint residuals = 7.542e-03
Evaluation 1377, objective function = -1.40

The solution vector from VMCON's solution doesn't work as an initial point in the original input file, whether using VMCON or scipy. The output doesn't work as an input!

## VMCON solution as initial point for no f-value IN.DAT

In [15]:
# Now read in original IN.DAT
in_dat_path = Path("../no_f_values_demo_IN.DAT")
in_dat = InDat(in_dat_path)

# Change to the new optimisation parameter values
for var_name, value in opt_params.items():
    # Filter out all f-beginning vars, except allowed iter vars!
    allowed_f_opt_params = ["fvsbrnni", "fcutfsu", "fimp(13)", "fgwped"]
    if var_name[0] != "f" or var_name in allowed_f_opt_params:
        in_dat.add_parameter(var_name, value)

# Write out new IN.DAT, with optimisation parameters set to previous local solution
new_in_dat_path = Path("no_f_values_demo_vmcon_soln_IN.DAT")
in_dat.write_in_dat(new_in_dat_path)

process.io.python_fortran_dicts - ERROR - Can't find the dicts JSON file
Traceback (most recent call last):
  File "/home/jon/anaconda3/envs/process/lib/python3.10/site-packages/process/io/python_fortran_dicts.py", line 27, in get_dicts
FileNotFoundError: [Errno 2] No such file or directory: '/home/jon/anaconda3/envs/process/lib/python3.10/site-packages/process/io/python_fortran_dicts.json'


FileNotFoundError: [Errno 2] No such file or directory: '/home/jon/anaconda3/envs/process/lib/python3.10/site-packages/process/io/python_fortran_dicts.json'

In [14]:
single_run = process.main.SingleRun(str(new_in_dat_path), solver="scipy")
single_run.run()

The IN.DAT file does not contain any obsolete variables.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 2.4.0   Release Date :: 2022-05-18
   Tag No. : v2.4.0-963-gcda1a8011 code contains untracked changes
    Branch : nlopt-soln-plotting
   Git log : Use iter var bounds in input.f90
 Date/time : 21 May 2023 16:07:36 +01:00(hh:mm) UTC
      User : jon
  Computer : jon-Precision-3560
 Directory : /home/jon/code/notebooks/global_opt/no_f_value_initial_test
     Input : /home/jon/code/notebooks/global_opt/no_f_value_initial_test/no_f_values_demo_vmcon_soln_IN.DAT
 R

/home/jon/anaconda3/envs/process/lib/python3.10/site-packages/process/sctfcoil.py:1648: RuntimeWarning: divide by zero encountered in divide
  tfcoil_variables.oacdcp = tfcoil_variables.ritfc / tfcoil_variables.tfareain
/home/jon/anaconda3/envs/process/lib/python3.10/site-packages/process/sctfcoil.py:3155: RuntimeWarning: divide by zero encountered in divide
  tfcoil_variables.n_tf
/home/jon/anaconda3/envs/process/lib/python3.10/site-packages/process/sctfcoil.py:3165: RuntimeWarning: divide by zero encountered in divide
  tfcoil_variables.n_tf * sctfcoil_module.a_tf_ins / tfcoil_variables.tfareain


ZeroDivisionError: division by zero